In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time
import csv
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from time import strptime

In [293]:
# take the company ticker input from the user, the list of compant tickers can be found at 

ticker =  input('Enter the company ticker: ') 

Enter the company ticker: AMZN


In [ ]:
driver = webdriver.Chrome()

In [295]:
driver.get('https://seekingalpha.com/symbol/'+ticker+'/news') # opens tickers page in selemiun test browser 

In [4]:
print(driver.title) # check to confirm that the correct page is opened 

Amazon.com, Inc. (AMZN) Latest Stock News | Seeking Alpha


In [5]:
# The page initially dispalys only 20 results, to get 20 more results, we have to move the scroll bar down
# Once 20 results are loaded we have to move scroll bar up and then come down to see further 20 results
# For 1,200 articles, we have to roughtly move the scroll bar, up and down for 60 times

for i in range(60):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # moving scroll bar down to get more articles
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 1000)") # scroll bar should be moved up before the next cycle
    time.sleep(2)
    print(i) # this counter tells us how many cycles have run so far

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [6]:
element = driver.find_elements_by_tag_name('article') # all article hyperlinks in tag names 'articles'

In [7]:
len(element) # to check the number of articles loaded in the page 

1220

In [8]:
lst_links = [] # empty list of links

for j in range(len(element)):
    # the xpath of the article links was in sequence 
    # only ....div/div[3]/article['+str(j+1)+']/div/...changes for each articles
    # j+1 as the first link starts from ..div/div[3]/article[1]/div....
    txt = driver.find_element_by_xpath('//*[@id="root"]/div/main/div[1]/div/div[3]/div/div[1]/div[2]/div/section/div/div/div[3]/article['+str(j+1)+']/div/div/div/a[1]')
    req_link = txt.get_attribute('href').split('?')[0] # getting hyperlink of the article and removing the redundent part of the hyderlink
    lst_links.append(req_link) # appending it to link of links
    print(j) # counter tells us how many links have been appended so far

In [9]:
len(lst_links) # to check the number of links in the list

1220

In [13]:
links_obj = pd.Series(lst_links)

In [15]:
links_obj.to_csv('amzn_links.csv') # saving the file

In [ ]:
article_lst = []

In [108]:
j = 0
for link in lst_links:
    driver.get(link)
    time.sleep(1) # Wait for a second so that the page loads
    date = driver.find_element_by_xpath('//*[@id="mc-hd"]/div[3]/time') # finds the date in string form
    author = driver.find_element_by_xpath('//*[@id="mc-hd"]/div[3]/span[3]/a') # finds the author of the page 
    about = driver.find_element_by_xpath('//*[@id="mc-hd"]/div[3]/a[1]') # finds the comapny name
    article = driver.find_element_by_xpath('//*[@id="a-cont"]') # finds the article
    article_lst.append([date.text, author.text, about.text, article.text]) # appends all the details of the article to the list
    print(j)
    # time.sleep(random.randrange(1, 3))
    # sometime seeking alpha asks us to verufy if we are robots. This loop will stop at that time
    j += 1 # this counter will help us know how many artickes have been appended and where to restart the loop when the website stops us

1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219


In [111]:
amzn_articles = pd.DataFrame(article_lst, columns = ['date', 'author', 'company', 'article']) 

In [119]:
amzn_articles.to_csv('amzn_articles.csv') # save articles file

In [ ]:
amzn_prices = pd.read_csv ('amzn_prices.csv')

In [5]:
amzn_articles = pd.read_csv ('amzn_articles.csv') # file downloaded from yahoo finance 

In [6]:
amzn_articles

,Unnamed: 0,date,author,company,article
0,0,"Jun. 22, 2020 2:36 PM ET",Jason Aycock,Microsoft Corporation (MSFT),Microsoft (MSFT +2.4%) is shutting down its Mi...
1,1,"Jun. 22, 2020 10:00 AM ET",Brad Olesen,"Sonos, Inc. (SONO)","The time may be right for that ""long waited"" a..."
2,2,"Jun. 20, 2020 8:35 AM ET",Clark Schultz,"Amazon.com, Inc. (AMZN)",Amazon (NASDAQ:AMZN) will get its start in del...
3,3,"Jun. 18, 2020 10:43 AM ET",Brandy Betz,"Amazon.com, Inc. (AMZN)",Valentino and Amazon (AMZN +0.1%) have filed a...
4,4,"Jun. 18, 2020 8:14 AM ET",Brandy Betz,"Amazon.com, Inc. (AMZN)",Amazon (NASDAQ:AMZN) is hiring in South Africa...
...,...,...,...,...,...
1199,1199,"Jun. 22, 2018 4:54 AM ET",Yoel Minkoff,"Amazon.com, Inc. (AMZN)",The NSA has moved most of its vast data resour...
1200,1200,"Jun. 21, 2018 10:36 AM ET",Clark Schultz,"Amazon.com, Inc. (AMZN)",The Supreme Court rules that states have the c...
1201,1201,"Jun. 21, 2018 10:24 AM ET",Brandy Betz,"Amazon.com, Inc. (AMZN)",Amazon (NASDAQ:AMZN) Prime Day could happen on...
1202,1202,"Jun. 20, 2018 10:58 AM ET",Liz Kiesche,JPMorgan Chase & Co. (JPM),J.P. Morgan (JPM +0.5%) Asset Management enter...


## Preparing data for analysis

In [188]:
# calculating the % change in the stock price during the day
pct_change = []
for i, price in enumerate(amzn_prices['Adj Close']):
    if i == 0:
        last_close = price # the fisrt day in the list of prices dosent have the previous closeing price
        change = 0.00 # hence change in price is 0
    else:
        change  = (price - last_close)/last_close # calculating the % change during the day 
        last_close = price
    pct_change.append(change)      

[0.0,
 -0.011547330916552743,
 -0.0016785908872026085,
 -0.004544421660380528,
 -0.018808538888721252,
 0.009840439090882227,
 0.007602154001264008,
 0.0007315576236918441,
 -0.0038152704629490727,
 0.0015594715759213054,
 -0.01607434737114308,
 0.010914598252326897,
 0.02099494621210366,
 0.027192164163329858,
 0.02195052318270812,
 -0.009407833267060298,
 0.030985079767622787,
 0.01583662577983729,
 0.010538075598060557,
 -0.00020488834003405185,
 0.0005532561475408868,
 -0.012472599457674196,
 0.09797169249201934,
 0.0037588116502962343,
 -0.010123904844367726,
 0.0056460103725370905,
 0.014669206049149274,
 -0.0011364222159544548,
 -0.0020889582203954486,
 -0.005850148272705611,
 0.009607042243712956,
 -0.014059305896364921,
 -0.01291887341023826,
 0.002640551477560637,
 0.006603095419847291,
 -0.0029387042341570005,
 0.007130497979736923,
 0.0035117756576255547,
 0.00696107552073004,
 -0.0003735865827770874,
 -0.003925155922202767,
 -0.032162340589131165,
 -0.04131611695263999,
 -

In [189]:
amzn_prices['pct_change'] = pct_change # adding a row for percentahe change 

In [203]:
f_date = []

for date in amzn_prices['Date']:
    f_date.append(datetime.strptime(date,'%Y-%m-%d').date()) # changing string date to datetime object
    

[datetime.date(2015, 6, 23),
 datetime.date(2015, 6, 24),
 datetime.date(2015, 6, 25),
 datetime.date(2015, 6, 26),
 datetime.date(2015, 6, 29),
 datetime.date(2015, 6, 30),
 datetime.date(2015, 7, 1),
 datetime.date(2015, 7, 2),
 datetime.date(2015, 7, 6),
 datetime.date(2015, 7, 7),
 datetime.date(2015, 7, 8),
 datetime.date(2015, 7, 9),
 datetime.date(2015, 7, 10),
 datetime.date(2015, 7, 13),
 datetime.date(2015, 7, 14),
 datetime.date(2015, 7, 15),
 datetime.date(2015, 7, 16),
 datetime.date(2015, 7, 17),
 datetime.date(2015, 7, 20),
 datetime.date(2015, 7, 21),
 datetime.date(2015, 7, 22),
 datetime.date(2015, 7, 23),
 datetime.date(2015, 7, 24),
 datetime.date(2015, 7, 27),
 datetime.date(2015, 7, 28),
 datetime.date(2015, 7, 29),
 datetime.date(2015, 7, 30),
 datetime.date(2015, 7, 31),
 datetime.date(2015, 8, 3),
 datetime.date(2015, 8, 4),
 datetime.date(2015, 8, 5),
 datetime.date(2015, 8, 6),
 datetime.date(2015, 8, 7),
 datetime.date(2015, 8, 10),
 datetime.date(2015, 8, 1

In [204]:
amzn_prices['f_date'] = f_date

In [1]:
amzn_articles

NameError: name 'amzn_articles' is not defined

In [135]:
copy = amzn_articles.copy() # creating a copy of df amzn_articles, so that we dont overwrite any crutial data

In [207]:
i = 0
f_date = []

for lst in copy['date'].str.split(): # spliting the date string into different parts before iterating over them 
    lst[0] = lst[0].replace('.','') 
    lst[1] = lst[1].replace(',','')
    if lst[4] == 'PM':              # converting the date in 24 hours format
        hour  = int(lst[3].split(':')[0]) + 12 # splitting time into hours and minutes 
        if hour == 24:             
            hour = 12         # 12 pm should be 12 in 24 hours format 
        minute  = int(lst[3].split(':')[1]) # splitting time into hours and minutes
    else:
        hour  = int(lst[3].split(':')[0])
        minute  = int(lst[3].split(':')[1])
    #f_date.append(strptime(lst[0],'%b').tm_mon)
    
    x  = datetime(int(lst[2]),int(strptime(lst[0],'%b').tm_mon),int(lst[1]),hour,minute) # converting into datetime object 
    
    f_date.append(x)
    
len(f_date) # to check if all data points got converted 
    

1204

In [208]:
amzn_articles['f_date_time'] = f_date

In [212]:
# creating a new row for just the date 

only_date = []

for d_tim in amzn_articles['f_date']:
    x = d_tim.date()
    only_date.append(x)
    
only_date  
    

[datetime.date(2020, 6, 22),
 datetime.date(2020, 6, 22),
 datetime.date(2020, 6, 20),
 datetime.date(2020, 6, 18),
 datetime.date(2020, 6, 18),
 datetime.date(2020, 6, 18),
 datetime.date(2020, 6, 17),
 datetime.date(2020, 6, 17),
 datetime.date(2020, 6, 17),
 datetime.date(2020, 6, 16),
 datetime.date(2020, 6, 16),
 datetime.date(2020, 6, 15),
 datetime.date(2020, 6, 14),
 datetime.date(2020, 6, 12),
 datetime.date(2020, 6, 12),
 datetime.date(2020, 6, 11),
 datetime.date(2020, 6, 11),
 datetime.date(2020, 6, 11),
 datetime.date(2020, 6, 10),
 datetime.date(2020, 6, 10),
 datetime.date(2020, 6, 10),
 datetime.date(2020, 6, 9),
 datetime.date(2020, 6, 9),
 datetime.date(2020, 6, 9),
 datetime.date(2020, 6, 8),
 datetime.date(2020, 6, 8),
 datetime.date(2020, 6, 8),
 datetime.date(2020, 6, 8),
 datetime.date(2020, 6, 4),
 datetime.date(2020, 6, 4),
 datetime.date(2020, 6, 4),
 datetime.date(2020, 6, 3),
 datetime.date(2020, 6, 3),
 datetime.date(2020, 6, 2),
 datetime.date(2020, 6, 2),

In [213]:
amzn_articles['f_date'] = only_date

## Joining articles df and prices df

In [222]:
final = pd.merge(amzn_articles,amzn_prices, on='f_date', how='left')

In [242]:
test  = final[['f_date','company', 'pct_change', 'article']] # we only need this data for analysis

In [243]:
test.fillna(method='bfill') # weekends should have next mondays prices for analysis

,f_date,company,pct_change,article
0,2020-06-22,Microsoft Corporation (MSFT),0.014508,Microsoft (MSFT +2.4%) is shutting down its Mi...
1,2020-06-22,"Sonos, Inc. (SONO)",0.014508,"The time may be right for that ""long waited"" a..."
2,2020-06-20,"Amazon.com, Inc. (AMZN)",NaN,Amazon (NASDAQ:AMZN) will get its start in del...
3,2020-06-18,"Amazon.com, Inc. (AMZN)",0.004922,Valentino and Amazon (AMZN +0.1%) have filed a...
4,2020-06-18,"Amazon.com, Inc. (AMZN)",0.004922,Amazon (NASDAQ:AMZN) is hiring in South Africa...
...,...,...,...,...
1199,2018-06-22,"Amazon.com, Inc. (AMZN)",-0.008409,The NSA has moved most of its vast data resour...
1200,2018-06-21,"Amazon.com, Inc. (AMZN)",-0.011348,The Supreme Court rules that states have the c...
1201,2018-06-21,"Amazon.com, Inc. (AMZN)",-0.011348,Amazon (NASDAQ:AMZN) Prime Day could happen on...
1202,2018-06-20,JPMorgan Chase & Co. (JPM),0.008820,J.P. Morgan (JPM +0.5%) Asset Management enter...


In [248]:
final  = test.fillna(method='ffill')

In [250]:
final.to_csv('amzn_final.csv')

In [275]:
# removing the ads at the bottom of the articles 
clean_article = []

for artl in final['article']:
    split_artl = artl.split('Now read:')
    clean_article.append(split_artl[0])
    
print(clean_article)

["Microsoft (MSFT +2.4%) is shutting down its Mixer live-streaming service, and will move its streamers and the audience to Facebook Gaming (FB +0.2%).\nThat's a setback for Microsoft's ambitions to compete with Twitch (NASDAQ:AMZN) and YouTube (GOOG, GOOGL), but looks likely to broaden its gaming service reach: Microsoft will provide its Project xCloud streaming service to Facebook Gaming.\nMicrosoft aimed at Twitch when it launched Mixer in 2017, after reconfiguring its Beam acquisition from the year before.\nAs of last August, Mixer had about 30M viewers.\nMicrosoft will retain Beam's technology, and use its low-latency video networking for internal purposes.\n", 'The time may be right for that "long waited" acquisition of Sonos (NASDAQ:SONO), Citron Research tweeted, with Apple (NASDAQ:AAPL) as a logical acquirer. \nThe speaker specialist is leader in the "stay at home" economy, Citron says, given that it is at the center of the "connected home. The co. is "dominating" home audio w

In [276]:
final['article'] = clean_article

In [280]:
# for analysis we need only articels related to amazon 
amzn_final  = final[final['company'] == 'Amazon.com, Inc. (AMZN)']

In [297]:
amzn_final.to_csv('amzn_final.csv')

In [282]:
# pushing data to find sentiment 

In [285]:
# finding positive articles, i.e. articles which resulted in positive price change
amzn_pv = amzn_final[amzn_final['pct_change'] > 0]

In [286]:
amzn_pv.to_csv('amzn_pv.csv')

In [288]:
# finding negetive articles, i.e. articles which resulted in negetive price change
amzn_nv = amzn_final[amzn_final['pct_change'] <= 0]

In [289]:
amzn_nv.to_csv('amzn_nv.csv')

In [ ]:
# skipped articles 11, 14, 30, 35, 47, 72, 99, 117, 172, 195, 221, 243, 247, 304, 311, 317, 435